### Imports

In [5]:
import os
import pandas as pd
import numpy as np

### Loading Data into Text File

In [9]:
folder = "/Users/reeteshsudhakar/Documents/GitHub/utk-image-classifier/utkface-images"

files = os.listdir(folder)
# grabbing the info from each file name and printing it  

# opening the text file
with open("image_data.txt", "w") as outfile:
    for file in files:
        file = file.strip(".jpg.chip.jpg")
        counter = 0
        for item in file.split("_"):
            outfile.write(item + "\n")
            counter += 1 

            if counter == 4:
                outfile.write("-----------\n")
                continue
    # TODO - modify this to grab the info and add it to a table 

### Image Labels
The labels of each face image is embedded in the file name, formated like [age]_[gender]_[race]_[date&time].jpg

**age** is an integer from 0 to 116, indicating the age

**gender** is either 0 (male) or 1 (female)

**race** is an integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern).

**date&time** is in the format of yyyymmddHHMMSSFFF, showing the date and time an image was collected to UTKFace

### Dataframe Testing

In [7]:
df2 = pd.DataFrame(
    {
        "Age": 1.0,
        "Gender": pd.Timestamp("20130102"),
        "Race": pd.Series(1, index=list(range(4)), dtype="float32"),
        "Date & Time": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)

df2

,Age,Gender,Race,Date & Time,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo
